In [33]:
import torch
import torch.nn as nn
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import InMemoryDataset
import pickle
import pandas as pd
import numpy as np

In [9]:
import torch.nn.utils.rnn as rnnutils

def unpack_sequence(packed_sequence, lengths):
    assert isinstance(packed_sequence, rnnutils.PackedSequence)
    head = 0
    trailing_dims = packed_sequence.data.shape[1:]
    unpacked_sequence = [torch.zeros(l, *trailing_dims) for l in lengths]
    # l_idx - goes from 0 - maxLen-1
    for l_idx, b_size in enumerate(packed_sequence.batch_sizes):
        for b_idx in range(b_size):
            unpacked_sequence[b_idx][l_idx] = packed_sequence.data[head]
            head += 1
    return unpacked_sequence

In [2]:
path = '../blockchain_files/3000000to3999999_BlockTransaction/'
with open(path+"graphs_data_rnn", "rb") as fp:   # Unpickling
    read_graph_data = pickle.load(fp)

In [103]:
data_list = []
for edges,edge_attributes,label in read_graph_data:
    if len(edge_attributes) == 0:
        continue
    data_list.append((edges,edge_attributes,label))

In [69]:
edges,edge_attributes,label = read_graph_data[0]

In [77]:
rnn = nn.RNN(4, 16, 1)
inputs = torch.randn(5, 3, 4)
h0 = torch.randn(1, 3, 16)
output, hn = rnn(inputs, h0)
print(output.shape)
print(hn.shape)

torch.Size([5, 3, 16])
torch.Size([1, 3, 16])


In [46]:
output, hn = rnn(edge_attributes.float())
hn.shape

torch.Size([1, 16584, 16])

In [83]:
def det_graph_data(edges,edge_attributes,label):
    pd_edges = pd.DataFrame(edges)
    pd_edges
    nodes = pd.concat([pd_edges[0]
                      ,pd_edges[1]]).unique()
    edge_index = torch.tensor(np.array(edges).T, dtype=torch.long)
    x = torch.ones(nodes.shape).reshape(-1,1)
    output, hn = rnn(edge_attributes.float().to(device))
    data = Data(x=x, edge_index=edge_index,edge_attr = hn[0],y=torch.tensor([label]))
    
    return data

In [84]:
det_graph_data(edges,edge_attributes,label)

Data(x=[47130, 1], edge_index=[2, 62918], edge_attr=[62918, 16], y=[1])

In [87]:
from typing import Tuple, Union

import torch
import torch.nn.functional as F
from torch import Tensor
from torch.nn import BatchNorm1d, Linear

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj, OptTensor, PairTensor


class CGConv(MessagePassing):

    def __init__(self, channels: Union[int, Tuple[int, int]], dim: int = 0,
                 aggr: str = 'mean', batch_norm: bool = False,
                 bias: bool = True, **kwargs):
        super().__init__(aggr=aggr, **kwargs)
        # print(channels)
        self.channels = channels
        self.dim = dim
        self.batch_norm = batch_norm

        if isinstance(channels, int):
            channels = (channels, channels)
        self.lin_f = Linear(2*channels[0] + dim, channels[1], bias=bias)
        # self.lin_s = Linear(2*channels[0] + dim, channels[1], bias=bias)
        if batch_norm:
            self.bn = BatchNorm1d(channels[1])
        else:
            self.bn = None

        self.reset_parameters()


    def reset_parameters(self):
        self.lin_f.reset_parameters()
        # self.lin_s.reset_parameters()
        if self.bn is not None:
            self.bn.reset_parameters()


    def forward(self, x: Union[Tensor, PairTensor], edge_index: Adj,
                edge_attr: OptTensor = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: PairTensor = (x, x)

        # propagate_type: (x: PairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=None)
        out = out if self.bn is None else self.bn(out)
        out += x[1]
        # print(out.shape)
        # print(x[1])
        return out


    def message(self, x_i, x_j, edge_attr: OptTensor) -> Tensor:
        if edge_attr is None:
            z = torch.cat([x_i, x_j], dim=-1)
        else:
#             print(x_i.shape)
#             print(x_j.shape)
#             print(edge_attr.shape)
            z = torch.cat([x_i, x_j, edge_attr], dim=-1)

#         print(z.shape)
#         print(z)
        # print(self.lin_f)
        return self.lin_f(z)

    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.channels}, dim={self.dim})'

In [88]:
conv1 = CGConv((1, 64),5,batch_norm=True)

In [97]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
#         torch.manual_seed(12345)
        self.conv1 = CGConv((1, hidden_channels),16,batch_norm=True)
#         self.conv1 = GCNConv(1, hidden_channels)
        
        self.conv2 = CGConv((hidden_channels, hidden_channels),16,batch_norm=True)
        self.conv3 = CGConv((hidden_channels, hidden_channels),16,batch_norm=True)
        self.conv4 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index,edge_attributes, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv2(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv3(x, edge_index,edge_attributes)
        x = x.relu()
        x = self.conv4(x, edge_index)
 

        # 2. Readout layer
#         print(x.shape)
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
#         print(x.shape)
        # 3. Apply a final classifier
#         x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

GCN(
  (conv1): CGConv((1, 64), dim=16)
  (conv2): CGConv((64, 64), dim=16)
  (conv3): CGConv((64, 64), dim=16)
  (conv4): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)


In [98]:
rnn.parameters()

<generator object Module.parameters at 0x7f8db436ca50>

In [154]:
rnn = nn.RNN(4, 16, 1,nonlinearity='relu').to(device)
model = GCN(hidden_channels=64).to(device)
net_params = list(model.parameters()) + list(rnn.parameters())
optimizer = torch.optim.Adam(net_params, lr=0.01)
# weights = torch.tensor([.05,.95]).to(device)
criterion = torch.nn.CrossEntropyLoss()

def train_mode():
    rnn.train()
    model.train()
train_mode()

In [155]:
def create_data_loader(n):
    indices = np.random.choice(len(data_list),5)
    
    graphs = []
    for i in indices:
        graph = data_list[i]
        graphs.append(det_graph_data(graph[0],graph[1],graph[2]))
        
    train_loader = DataLoader(graphs, batch_size=n,shuffle=True)
    
    return train_loader

In [156]:
loader = create_data_loader(3)
data = next(iter(loader))
# data.y
data = data.to(device)
# data.edge_attr

tensor([[1.3238, 0.0000, 0.0000,  ..., 0.5141, 0.7933, 0.0000],
        [0.8585, 0.0000, 0.0000,  ..., 0.5308, 0.5185, 0.0000],
        [0.9695, 0.0000, 0.0000,  ..., 0.2996, 0.6464, 0.0000],
        ...,
        [6.4548, 0.0000, 1.7951,  ..., 3.2290, 0.6063, 0.0000],
        [1.6110, 0.0000, 0.0000,  ..., 0.5430, 0.2932, 0.0000],
        [3.2221, 0.0000, 0.4784,  ..., 1.4941, 0.2342, 0.0000]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [132]:
np.random.choice(10,2).tolist()

[7, 8]

In [157]:
def train():
    for i in range(len(data_list)):

        train_loader = create_data_loader(10)
        
        data = next(iter(train_loader))
#         print(data.edge_attr)
        data = data.to(device)
        out = model(data.x, data.edge_index,data.edge_attr,data.batch)
        
        loss = criterion(out, data.y)
        
        loss.backward()  # Derive gradients.
        
        optimizer.step()
        optimizer.zero_grad()
#         break
    return loss.cpu().item()
# train()

In [158]:

for epoch in range(1, 400):
    loss = train()
    print(loss)

nan


KeyboardInterrupt: 

In [ ]:
def train():
    model.train()
    for batch_idx,data in enumerate(train_loader):


        data.edge_attr = data.edge_attr.float()
        data = data.to(device)
        out = model(data.x, data.edge_index,data.edge_attr,data.batch)

        loss = criterion(out, data.y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        
        optimizer.step()
        optimizer.zero_grad()
#             print(batch_idx)
        

    return loss.cpu().item()
    
def test(loader):
    model.eval()

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        


        data.edge_attr = data.edge_attr.float() 
        data = data.to(device)
        out = model(data.x, data.edge_index,data.edge_attr, data.batch)  
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        

        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.

test(test_loader)

In [ ]:

for epoch in range(1, 400):
    loss = train()
#     break
    train_acc = test(test_loader)
#     test_acc = test(test_loader)
    if epoch % 5 == 0: 
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, loss : {loss:.4f}')

In [ ]:
train_acc = test(test_loader)

In [ ]:
train_acc

In [ ]:
import numpy as np
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([p.numel() for p in model_parameters])
params

In [ ]:
read_graph_data[0]